In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = '',
    model="llama-3.3-70b-versatile",
)

response = llm.invoke("The first person to land on the moon was..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [21]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.wipro.com/job/RPA-Business-Analyst/98144-en_US")
page_data = loader.load().pop().page_content
print(page_data)













RPA Business Analyst Job Details | Wipro Limited



























We use cookies to offer you the best possible website experience. Your cookie preferences will be stored in your browser’s local storage. This includes cookies necessary for the website's operation. Additionally, you can freely decide and change any time whether you accept cookies or choose to opt out of cookies to improve website's performance, as well as cookies used to display content tailored to your interests. Your experience of the site and the services we are able to offer may be impacted if you do not accept all cookies.

                Modify Cookie Preferences


                Accept All Cookies






Skip to main content




















Language 


العَرَبِيَّة (السعودية)


English (United States)


Español (España)


Español (México)


Français (Canada)


Français (France)


Italiano (Italia)


Polski


Português (Brasil)


简体中文 (中国大陆)





Login

































Ho

In [22]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
"""
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "RPA Business Analyst",
  "experience": "4+ years",
  "skills": [
    "Lucid chart",
    "Draw.io",
    "Copilot",
    "RPA",
    "Process mapping",
    "Engineering/RPA Background"
  ],
  "description": "The purpose of the role is to liaison and bridging the gap between customer and Wipro delivery team to comprehend and analyze customer requirements and articulating aptly to delivery teams thereby, ensuring right solutioning to the customer."
}
```


In [23]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'RPA Business Analyst',
 'experience': '4+ years',
 'skills': ['Lucid chart',
  'Draw.io',
  'Copilot',
  'RPA',
  'Process mapping',
  'Engineering/RPA Background'],
 'description': 'The purpose of the role is to liaison and bridging the gap between customer and Wipro delivery team to comprehend and analyze customer requirements and articulating aptly to delivery teams thereby, ensuring right solutioning to the customer.'}

In [24]:
type(json_res)

dict

In [25]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [26]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [27]:
job = json_res
job['skills']

['Lucid chart',
 'Draw.io',
 'Copilot',
 'RPA',
 'Process mapping',
 'Engineering/RPA Background']

In [28]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])

In [29]:
job = json_res
job['skills']

['Lucid chart',
 'Draw.io',
 'Copilot',
 'RPA',
 'Process mapping',
 'Engineering/RPA Background']

In [30]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [31]:
job

{'role': 'RPA Business Analyst',
 'experience': '4+ years',
 'skills': ['Lucid chart',
  'Draw.io',
  'Copilot',
  'RPA',
  'Process mapping',
  'Engineering/RPA Background'],
 'description': 'The purpose of the role is to liaison and bridging the gap between customer and Wipro delivery team to comprehend and analyze customer requirements and articulating aptly to delivery teams thereby, ensuring right solutioning to the customer.'}

In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Affan, a Data Associate at Edutan. Edutan is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Edutan 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Edutan's portfolio: {link_list}
        Remember you are Affan, Data Associate at Edutan. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert RPA Solutions for Seamless Business Process Integration

Dear Hiring Manager,

I came across the job description for an RPA Business Analyst at your esteemed organization, and I am excited to introduce Edutan, a leading AI & Software Consulting company, as a potential partner in fulfilling your RPA needs. With our expertise in automating business processes, we are confident that our tailored solutions can empower your enterprise to achieve scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team at Edutan possesses extensive experience in RPA, process mapping, and engineering, making us an ideal fit for your requirements. We have a proven track record of successfully implementing RPA solutions using tools like Lucid chart, Draw.io, and Copilot, ensuring seamless integration with your existing systems. Our expertise in liaising between customers and delivery teams enables us to comprehend and analyze customer requirements, articulat